In [8]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision
import torchvision.transforms as transforms
import numpy as np
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json

from sklearn.metrics import confusion_matrix
from PIL import Image

In [9]:
#from google.colab import drive
#drive.mount('/content/drive')
#

In [10]:
# Mappings for utility
MAP_PERCENT_TO_AMOUNT = {
    "10" : 280,
    "25" : 700,
    "50" : 1400,
    "100": 2799
}

DAMAGE_LEVEL_TO_SCORE = {
    "destroyed" : 4,
    "major-damage" : 3,
    "minor-damage" : 2,
    "no-damage" : 1,
}

PERCENT = "100"
TRAIN_SET_SIZE = MAP_PERCENT_TO_AMOUNT[PERCENT]

In [11]:
os.listdir()

[]

In [18]:
DATASET_FOLDER_PATH = "/datasets/xviewdataset/HTCV-Sem/dataset/"
XVIEW2_TXT_FILE = "/datasets/xviewdataset/HTCV-Sem/dataset/xview2.txt"
MODEL_OUTPUT_FILENAME = "model_deepnote_test"

In [13]:
!ls /datasets/xviewdataset/HTCV-Sem/dataset/

hold  test  train  xview2.txt


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Baseline model
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ResNetClassifier, self).__init__()
        self.resnet = models.resnet18(weights=None)
        self.resnet.fc = nn.Linear(512, num_classes)

        

    def forward(self, x):
        x = self.resnet(x)
        return x
    
    def train_model(self, dataloader, num_epochs, lr, momentum):
        self.train()
        self.to(device)

        optimizer = torch.optim.SGD(self.parameters(), lr, momentum)
        criterion = nn.CrossEntropyLoss()
        # Train the model

        for epoch in range(num_epochs):
            running_loss = 0.0
            for images, labels in dataloader:
                images, labels =  images.to(device),labels.to(device)
                
                optimizer.zero_grad()

                # Forward pass
                outputs = self(images)
                loss = criterion(outputs, labels)

                # Backward pass and optimizatio.n
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}")
        torch.save(self.state_dict(), MODEL_OUTPUT_FILENAME + '.pth')
    # Define the evaluation function
    def evaluate_model(self, test_dataloader):
        self.eval()  # Set the model to evaluation mode
        self.to(device)

        correct = 0
        total = 0

        true_labels = []
        predicted_labels = []

        with torch.no_grad():
            for images, labels in test_dataloader:
                images = images.to(device)
                labels = labels.to(device)

                outputs = self(images)
                _, predicted = torch.max(outputs.data, 1)

                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                true_labels.extend(labels.cpu().numpy())
                predicted_labels.extend(predicted.cpu().numpy())

        accuracy = correct / total
        confusion_mat = confusion_matrix(true_labels, predicted_labels, normalize="true")


        return accuracy, confusion_mat

In [15]:
# Get predefined training samples for the seminar
def get_training_set_entries(amount=280):
    xview_file = open(XVIEW2_TXT_FILE,'r')
    return xview_file.read().splitlines()[:amount]

In [16]:
# Get mean and standard deviation of dataset
# This is needed for normalization
def calc_mean_and_std_of_dataset(loader):
    mean = 0.0
    std = 0.0
    total_samples = 0

    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_samples += batch_samples

    mean /= total_samples
    std /= total_samples
    
    return mean, std

Normalize Dataset

In [17]:
# Load and preprocess your dataset using torchvision.transforms
pre_norm_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the input image to match the expected size of ResNet (224x224)
    transforms.ToTensor()
])
# Calculate the mean and standard deviation of your dataset
path_to_dataset = os.path.join(DATASET_FOLDER_PATH,"train")
dataset = torchvision.datasets.ImageFolder(path_to_dataset, transform=pre_norm_transform)
data_point_names = get_training_set_entries(amount = TRAIN_SET_SIZE)

subset = [dataset[dataset.imgs.index((file_path, class_label))] for file_path, class_label in dataset.imgs if os.path.basename(file_path) in data_point_names]
loader = torch.utils.data.DataLoader(subset, batch_size=32, shuffle=False)

mean, std = calc_mean_and_std_of_dataset(loader=loader)

Setup training model

In [ ]:


# Update the normalization transform using your dataset's mean and standard deviation
transform = transforms.Compose([
    pre_norm_transform,
    transforms.Normalize(mean, std)
])

# Example dataset loading using torchvision.datasets.ImageFolder
dataset = torchvision.datasets.ImageFolder(path_to_dataset, transform=transform)
subset = [dataset[dataset.imgs.index((file_path, class_label))] for file_path, class_label in dataset.imgs if os.path.basename(file_path) in data_point_names]
dataloader = torch.utils.data.DataLoader(subset, batch_size=32, shuffle=True)

Train model

In [ ]:
# Train the model
num_classes = 5  # Number of output classes

num_epochs=50
lr=0.01
momentum=0.9
model = ResNetClassifier(num_classes)

model.train_model(dataloader, num_epochs, lr, momentum)


Setup evaluating test performance

In [ ]:
test_dataset = torchvision.datasets.ImageFolder(os.path.join(DATASET_FOLDER_PATH, "test"), transform=transform)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False,num_workers=2, pin_memory=True)

Evaluate model on test data

In [ ]:

LOAD_MODEL = False
LOAD_MODEL_PATH = 'resnet_classifier.pth'
#Load the saved model state dictionary
if(LOAD_MODEL):
  model = ResNetClassifier(num_classes)
  model.load_state_dict(torch.load('resnet_classifier.pth'))

# Evaluate the model
accuracy, confusion_mat = model.evaluate_model(test_dataloader=test_dataloader)

# Print the accuracy
print(f"Accuracy: {accuracy}")

print("Confusion Matrix:")
print(confusion_mat)


In [ ]:
mean_val = [mean_i.item() for mean_i in mean]
std_val = [std_i.item() for std_i in std]
data_info = {
    'dataset_size' : MAP_PERCENT_TO_AMOUNT[PERCENT],
    'mean_0' : mean.[0],
    'mean_1' : mean_val[1],
    'mean_2' : mean_val[2],
    'standard_dev_0' : std_val[0],
    'standard_dev_1' : std_val[1],
    'standard_dev_2' : std_val[2]
}
model_info = {
    'dataset_size' : MAP_PERCENT_TO_AMOUNT[PERCENT],
    'architecture' : "resnet18",
    'learning_rate': lr,
    'batch_size': 32,
    'num_epochs': num_epochs,
    'momentum' : momentum,
    'accuracy' : accuracy,
    'confusion_mat' : confusion_mat.tolist()
}
with open("dataset_size_" + str(MAP_PERCENT_TO_AMOUNT[PERCENT]) +'.json', 'w') as file:
    json.dump(data_info, file)

with open(MODEL_OUTPUT_FILENAME +'.json', 'w') as file:
    json.dump(model_info, file)

Visualize Results

In [ ]:
# Get the class labels
class_labels = test_dataset.classes

# Create a figure and axis
fig, ax = plt.subplots(figsize=(8, 6))

# Plot the confusion matrix
sns.heatmap(confusion_mat, annot=True, fmt=".3f", cmap="Blues", cbar=False,
            xticklabels=class_labels, yticklabels=class_labels, ax=ax)

# Set labels and title
ax.set_xlabel("Predicted Labels")
ax.set_ylabel("True Labels")
ax.set_title("Confusion Matrix")

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Display the plot
plt.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ac044641-96a0-4e06-ae59-4421e551585a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>